In [26]:
import json
import csv

## Extração

O ````Open```` é uma função built-in, isto é, uma função que já vem incluída no Python. O primeiro parâmetro é o local onde está o o arquivo e como segundo parâmetro o modo que queremos abrir esse arquivo. Nesse caso, escolhemos 'r' (de read, leitura em inglês). Além disso, a a biblioteca JSON é utilizada e também já é padrão da linguagem.

In [35]:
path_json = r'C:\Users\je7560\Downloads\matheus\pipeline-dados\src\data\raw\dados_empresaA.json'

In [36]:
with open(path_json,'r') as file:
    data_json = json.load(file)

In [37]:
print(f"O data é do tipo: {type(data_json)}")

O data é do tipo: <class 'list'>


Já o arquivo CSV, os dados são separados por vírgulas, por isso que é um CSV. CVS significa, do inglês Comma Separated Values (Valores Separados por Vírgula). E da maneira que o readlines funciona, ele não consegue identificar isso. O ````csv.reader(file,delimiter=',')```` faz a leitura, mas não salva em uma estrutura, deixando essa responsabilidade para o usuário. Esse comando retorna uma lista de listas, como desejamos construir um pipeline de dados, é interessante que construamos ambos os dados numa única estrutura. Estas duas estruturas diferentes se tornam um problema quando queremos criar manipulações para ambos os arquivos. Usaremos a estrutura de dicionário e é dado por 
````
csv.DictReader(file,delimiter=',')
````

In [41]:
path_csv = r'C:\Users\je7560\Downloads\matheus\pipeline-dados\src\data\raw\dados_empresaB.csv'

In [42]:
data_csv = []
with open(path_csv,'r') as file:
    spamreader = csv.DictReader(file,delimiter=',')
    for row in spamreader:
        data_csv.append(row)

In [43]:
data_csv[0]

{'Nome do Item': 'LÃ¡pis de sobrancelha',
 'ClassificaÃ§Ã£o do Produto': 'Roupas',
 'Valor em Reais (R$)': '55.17',
 'Quantidade em Estoque': '62',
 'Nome da Loja': 'Filial 1',
 'Data da Venda': '2023-04-13 18:58:06.794203'}

## Transformação

Reparamos que os arquivos possuiem campos com nomes diferentes, o que era esperado, pois são dados de duas empresas diferentes. Em algum momento, cada empresa definiu o nome do produto. Nesse caso, é necessário definir apenas uma termo. Uma sugestão é:
"procurar a equipe de analytics e disponibilizar as informações, pedindo para escolherem os nomes das colunas que fazem mais sentido para eles."

In [38]:
columns_json = list(data_json[0].keys())
columns_csv = list(data_csv[0].keys())

In [39]:
print(f"{columns_json}")
print(f"{columns_csv}")

['Nome do Produto', 'Categoria do Produto', 'Preço do Produto (R$)', 'Quantidade em Estoque', 'Filial']
['Nome do Item', 'ClassificaÃ§Ã£o do Produto', 'Valor em Reais (R$)', 'Quantidade em Estoque', 'Nome da Loja', 'Data da Venda']


### Alterando campo da tabela

In [40]:
key_mapping = {'Nome do Item': 'Nome do Produto',
                'ClassificaÃ§Ã£o do Produto': 'Categoria do Produto',
                'Valor em Reais (R$)': 'Preço do Produto (R$)',
                'Quantidade em Estoque': 'Quantidade em Estoque',
                'Nome da Loja': 'Filial',
                'Data da Venda': 'Data da Venda'}
key_mapping

{'Nome do Item': 'Nome do Produto',
 'ClassificaÃ§Ã£o do Produto': 'Categoria do Produto',
 'Valor em Reais (R$)': 'Preço do Produto (R$)',
 'Quantidade em Estoque': 'Quantidade em Estoque',
 'Nome da Loja': 'Filial',
 'Data da Venda': 'Data da Venda'}

In [46]:
transf_datacsv = []
for old_dict in data_csv:
    dict_temp = {}
    for old_key, value in old_dict.items():
        dict_temp[key_mapping[old_key]] = value
    transf_datacsv.append(dict_temp)


In [47]:
name_columns = list(transf_datacsv[0].keys())
name_columns

['Nome do Produto',
 'Categoria do Produto',
 'Preço do Produto (R$)',
 'Quantidade em Estoque',
 'Filial',
 'Data da Venda']

In [48]:
combina_list = []
combina_list.extend(data_json)
combina_list.extend(transf_datacsv)
len(combina_list)

4446

### Adicionando valores nos campos nulos

In [54]:
list(combina_list[0].keys())

['Nome do Produto',
 'Categoria do Produto',
 'Preço do Produto (R$)',
 'Quantidade em Estoque',
 'Filial']

In [57]:
columns_name = list(combina_list[-1].keys())
columns_name

['Nome do Produto',
 'Categoria do Produto',
 'Preço do Produto (R$)',
 'Quantidade em Estoque',
 'Filial',
 'Data da Venda']

In [59]:
dados_combinados_tabela = [columns_name]

for row in combina_list:
    linha = []
    for coluna in columns_name:
        linha.append(row.get(coluna, 'Indisponivel'))
    dados_combinados_tabela.append(linha)


In [60]:
dados_combinados_tabela

[['Nome do Produto',
  'Categoria do Produto',
  'Preço do Produto (R$)',
  'Quantidade em Estoque',
  'Filial',
  'Data da Venda'],
 ['Blush em pó', 'Eletrodomésticos', 79.41, 7, 'Filial 7', 'Indisponivel'],
 ['Lápis de sobrancelha',
  'Eletrodomésticos',
  85.47,
  78,
  'Filial 8',
  'Indisponivel'],
 ['Base líquida', 'Roupas', 75.02, 34, 'Filial 8', 'Indisponivel'],
 ['Base líquida', 'Roupas', 44.94, 90, 'Filial 10', 'Indisponivel'],
 ['Lápis de sobrancelha', 'Alimentos', 58.63, 91, 'Filial 7', 'Indisponivel'],
 ['Blush em pó', 'Roupas', 50.79, 77, 'Filial 7', 'Indisponivel'],
 ['Corretivo', 'Eletrodomésticos', 20.46, 2, 'Filial 4', 'Indisponivel'],
 ['Pó compacto', 'Roupas', 85.56, 37, 'Filial 9', 'Indisponivel'],
 ['Sombra de olhos', 'Roupas', 31.92, 56, 'Filial 8', 'Indisponivel'],
 ['Delineador líquido', 'Alimentos', 42.58, 8, 'Filial 3', 'Indisponivel'],
 ['Corretivo', 'Eletrodomésticos', 92.03, 10, 'Filial 8', 'Indisponivel'],
 ['Lápis de sobrancelha', 'Roupas', 91.01, 95, 'F

## Carregar

In [61]:
path_data = r'C:\Users\je7560\Downloads\matheus\pipeline-dados\src\data\ready-processed\data.csv'

In [63]:
with open(path_data,'w') as file:
    whiter = csv.writer(file)
    
    whiter.writerows(dados_combinados_tabela)